In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson
from selenium.webdriver.chrome.options import Options

In [2]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    response.encoding = 'utf-8'
    return response

In [3]:
def getResponseNotForText(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    #response.encoding = 'utf-8'
    return response

In [4]:
# 获取m3u8文件地址
def getM3U8(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(options=chrome_options)
    #browser = webdriver.Chrome()
    browser.get(url)
    html = browser.page_source
    tree = etree.HTML(html)
    sourseUrl = tree.xpath('//*[@id="myFlash_player_html5_api"]/source/@src')
    if len(sourseUrl) == 0:
        return ""
    return sourseUrl[0]

In [5]:
print(getM3U8('https://tv.cctv.com/2012/09/21/VIDE1348164023130421.shtml?spm=C22284.P87020348575.ED2e9NOT2fLD.169'))

https://dh5.cntv.lxdns.com/asp/h5e/hls/main/0303000a/3/default/664254f3431146608cbbb87bf57b9adb/main.m3u8?maxbr=2048&contentid=15120519184043


In [6]:
def downloadTS(url):
    #  https://blog.csdn.net/liujiayu2/article/details/86083400
    # 下载M3U8文件中所有片段
    # https://dh5.cntv.kcdnvip.com/asp/h5e/hls/1200/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/1200.m3u8
    # 需要通过视频页面的m3u8地址下载到的m3u8文件获取事及m3u8文件
    #url = 'https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/main.m3u8?maxbr=2048&contentid=15120519184043'
    url = getM3U8(url)# 获取main M3U8 的url
    if url == "":
        print("error")
        return
    print('main M3U8: ', url)
    downloadPath = os.getcwd() + '/download'
    if not os.path.exists(downloadPath):
        os.mkdir(downloadPath)
    allContent = getResponse(url).text# 获取M3U8的文件内容
    fileLine = allContent.split('\n')# 读取文件里的每一行
    print(fileLine)
    m3u8Url = 'https://dh5.cntv.kcdnvip.com' + fileLine[-2]# 实际视频的M3U8的URL
    print("实际M3U8: ", m3u8Url)
    allContent = getResponse(m3u8Url).text
    fileLine = allContent.split('\n')
    
    # 通过判断文件头来确定是否是M3U8文件
    if fileLine[0] != "#EXTM3U":
        raise BaseException(u"非M3U8的链接")
    else:
        unknow = True# 用来判断是否找到了下载地址
        for index, line in enumerate(fileLine):
            if "EXTINF" in line:
                unknow = False
                # 拼出ts片段的URL
                # https://dh5.cntv.kcdnvip.com/asp/h5e/hls/1200/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/16.ts
                tsUrl = m3u8Url.rsplit('/', 1)[0] + '/' + fileLine[index + 1]# i.ts在下一行
                #print(tsUrl)
                res = getResponseNotForText(tsUrl)
                cFuleName = str(fileLine[index + 1])
                with open(downloadPath + '/' + cFuleName, 'ab') as fp:
                    fp.write(res.content)
                    fp.flush()
        if unknow:
            raise BaseException(u"未找到对应的下载地址")
        else:
            print("ts 下载完成")

In [7]:
# ts文件合并
def file_walker(path):
    file_list = []
    for root, dirs, files in os.walk(path): # 生成器
        for fn in files:
            p = str(root+'/'+fn)
            file_list.append(p)
 
    #print(file_list)
    return file_list
 
def combine(ts_path, combine_path, file_name):
    file_list = file_walker(ts_path)
    file_path = combine_path + file_name + '.ts'
    with open(file_path, 'wb+') as fw:
        for i in range(len(file_list)): 
            fw.write(open(file_list[i], 'rb').read())

In [8]:
# 下载并合成视频
def downloadAndCombine(url):
    fileName = (url.split('/')[-1]).split('.')[0]
    downloadTS(url)
    combine('./download', 'G:/work/cctv/对话/video/', fileName)
    print(fileName, " successful")

In [10]:
# https://tv.cctv.com/2012/09/21/VIDE1348164023130421.shtml?spm=C22284.P87020348575.ED2e9NOT2fLD.169
url = 'https://tv.cctv.com/2012/09/21/VIDE1348164023130421.shtml?spm=C22284.P87020348575.ED2e9NOT2fLD.169'
downloadAndCombine(url)

main M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/664254f3431146608cbbb87bf57b9adb/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/664254f3431146608cbbb87bf57b9adb/450.m3u8', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=870400, RESOLUTION=480x360', '/asp/h5e/hls/850/0303000a/3/default/664254f3431146608cbbb87bf57b9adb/850.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/850/0303000a/3/default/664254f3431146608cbbb87bf57b9adb/850.m3u8
ts 下载完成


FileNotFoundError: [Errno 2] No such file or directory: 'F:/work/cctv/对话/video/VIDE1348164023130421.ts'

In [9]:
# 删除之前下载的一段段的ts文件
def clear():
    shutil.rmtree('./download')
    time.sleep(1)
    os.mkdir('download')

In [10]:
# # 获取每页的所有视频链接
# 使用demjson
# 涉及将 application/x-javascript转为json
def getVideoUrls(url):
    page_text = getResponse(url).text
    jsonObj = demjson.decode(page_text[42:-2])
    linkList = []
    for obj in jsonObj['data']['list']:
        linkList.append(obj['url'])
    return linkList

In [21]:
# http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451533652476962&sort=desc&serviceId=tvcctv&mode=0&n=60&p=3&t=jsonp&cb=setItemByidELMTiyWjMLGGZU1i6mQ64DSQ190530
for link in getVideoUrls('http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451533652476962&sort=desc&serviceId=tvcctv&mode=0&n=60&p=3&t=jsonp&cb=setItemByidELMTiyWjMLGGZU1i6mQ64DSQ190530'):
    print(link)

https://tv.cctv.com/2020/10/09/VIDEWZUCk2YfAPE07qfHZkFv201009.shtml
https://tv.cctv.com/2020/10/07/VIDE0AxE0NqhiTp2qXawhT4m201007.shtml
https://tv.cctv.com/2020/10/07/VIDEMVlZxm3D2wVfTqWHi74F201007.shtml
https://tv.cctv.com/2020/10/05/VIDE4VK6iHiQwPYNr2NjxMSi201005.shtml
https://tv.cctv.com/2020/10/02/VIDEAz7eIxpSjhWUZ9DgLX1Y201002.shtml
https://tv.cctv.com/2020/11/02/VIDEFouYEt2JTSvv444NRV3Z201102.shtml
https://tv.cctv.com/2020/09/30/VIDEFvVWzH11UfqVLLGgzcUp200930.shtml
https://tv.cctv.com/2020/09/29/VIDEr8mSBYS2zlEze7TlhGEW200929.shtml
https://tv.cctv.com/2020/09/28/VIDEpkvfibWlLoP9geNXrprH200928.shtml
https://tv.cctv.com/2020/09/25/VIDEOCwN9h6hr0uM4IijZEkt200925.shtml
https://tv.cctv.com/2020/09/24/VIDEZVgRK0FFHxRPvLdDecul200924.shtml
https://tv.cctv.com/2020/09/23/VIDEjWmpI550fGzv7lRlTG0g200923.shtml
https://tv.cctv.com/2020/09/22/VIDEsExBUrSE66N23dRdddFk200922.shtml
https://tv.cctv.com/2020/09/21/VIDEAkXVqa31VnwXwm6AMsRB200921.shtml
https://tv.cctv.com/2020/09/19/VIDEaJc5lMikfd3fN

In [11]:
clear()

In [12]:
j = 0
for i in range(1, 7):
    url = f'http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451530382483536&sort=desc&serviceId=tvcctv&mode=0&n=60&p={i}&t=jsonp&cb=setItemByidELMTD2e9NOT2fLD60Bm0nJ5J190530'
    print(f'页面 {i}', url)
    #cnt = 0
    for link in getVideoUrls(url):
        if j < 128:
            j = j + 1
            continue
        j = j + 1
        print(f'video {j}')
        print('开始下载：', link)
        downloadAndCombine(link)
        clear()

页面 1 http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451530382483536&sort=desc&serviceId=tvcctv&mode=0&n=60&p=1&t=jsonp&cb=setItemByidELMTD2e9NOT2fLD60Bm0nJ5J190530
页面 2 http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451530382483536&sort=desc&serviceId=tvcctv&mode=0&n=60&p=2&t=jsonp&cb=setItemByidELMTD2e9NOT2fLD60Bm0nJ5J190530
页面 3 http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451530382483536&sort=desc&serviceId=tvcctv&mode=0&n=60&p=3&t=jsonp&cb=setItemByidELMTD2e9NOT2fLD60Bm0nJ5J190530
video 129
开始下载： https://tv.cctv.com/2018/09/04/VIDEWxwj3JIsJL2gegUbpDm9180904.shtml
main M3U8:  https://dh5.cntv.lxdns.com/asp/h5e/hls/main/0303000a/3/default/fa15fe9fcbec42529ff050b01d77e24d/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x270', '/asp/h5e/hls/450/0303000a/3/default/fa15fe9fcbec42529ff050b01d77e24d/450.m3u8', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=870400, RESOLUTION=640x360', '/a

KeyboardInterrupt: 